In [58]:
from Bio import SeqIO
from Bio import SeqFeature
from Bio.Alphabet import IUPAC
import pandas as pd
from Levenshtein import distance
import numpy as np

In [ ]:
data = ""

In [55]:
Baboons = list(SeqIO.parse("data/Baboons (sample).fasta", "fasta"))

In [56]:
for b in Baboons:
    location = b.description.split(' ')[1].split(':')
    location = location[1].split('-')
    location = SeqFeature.FeatureLocation(int(location[0]),int(location[1]))
    b.features.append(location)
    b.seq.alphabet = IUPAC.unambiguous_dna

In [ ]:
for sequence1 in first_species:
    matches = [distance(sequence1, sequence2) for sequence2 in second_species]
    print(np.argmax(matches))